## Build an End-to-End System
This puts together the chain of prompts that you saw throughout the course.

In [13]:
import os
import utils
import openai
from dotenv import load_dotenv, find_dotenv


from ipywidgets import Text, Button, Output, VBox, HTML
from IPython.display import display

# Create input widget
inp = Text(
    value='',
    placeholder='Type your message here',
    description='You:',
    disabled=False
)

# Optional: Output area
out = Output()

# Display the input field
display(VBox([inp, out]))


# Load environment variables from .env file
load_dotenv(find_dotenv())

# Initialize the OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Example: Chat completion
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! Can you recommend a product?"}
    ]
)

print(response.choices[0].message.content)


Of course! What type of product are you looking for?


In [ ]:
def get_completion_from_messages(messages, model="gpt-4", temperature=0.7, max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content

### System of chained prompts for processing the user query

In [ ]:
# utils.py

def find_category_and_product_only(user_input, product_list):
    # Minimal: just echo the input and product_list for demo
    # You can replace this with your real extraction logic
    return f"{user_input} | {product_list}"

def get_products_and_category():
    # Minimal: return a dummy list (replace with your real data if needed)
    return ["smartx pro phone", "fotosnap camera", "dslr", "tv", "tvs"]

def read_string_to_list(category_and_product_response):
    # Minimal: just wrap the response in a list for demo
    return [category_and_product_response]

def generate_output_string(category_and_product_list):
    # Minimal: join the list into a string for demo
    return "\n".join(str(x) for x in category_and_product_list)

In [ ]:
# main.py


def get_completion_from_messages(messages):
    # Get the latest user message
    user_message = ""
    for msg in reversed(messages):
        if msg['role'] == 'user':
            user_message = msg['content']
            break

    # Simple keyword-based logic
    user_message_lower = user_message.lower()
    if "hi" in user_message_lower or "hello" in user_message_lower:
        return "Hello! How can I assist you today?"
    elif "phone" in user_message_lower:
        return "We offer several phones, including the SmartX Pro and TCL 503. Would you like more details?"
    elif "camera" in user_message_lower:
        return "Our camera selection includes the FotoSnap DSLR and FotoSnap Compact. Interested in specs or pricing?"
    elif "tvs" in user_message_lower or "tv" in user_message_lower:
        return "We have a range of TVs, including TCL and Samsung models. What size are you looking for?"
    elif "thank" in user_message_lower:
        return "You're welcome! If you have any more questions, just ask."
    elif "bye" in user_message_lower:
        return "Goodbye! Have a great day."
    else:
        return "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."



def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    # For demo, we'll mock moderation (replace with real API in production)
    moderation_output = {"flagged": False}
    # response = openai.Moderation.create(input=user_input)
    # moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(
        user_input, utils.get_products_and_category()
    )
    # Step 2: Extract the list of products
    category_and_product_list = utils.read_string_to_list(category_and_product_response)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug: print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
    moderation_output = {"flagged": False}
    # response = openai.Moderation.create(input=final_response)
    # moderation_output = response["results"][0]

    if moderation_output["flagged"]:
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response, _ = process_user_message(user_input, [])
print(response)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model disapproved the response.
I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance.


### Function that collects user and assistant messages over time

In [ ]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
   

### Chat with the chatbot!¶
Note that the system message includes detailed instructions about what the OrderBot should do.

In [ ]:
# filename: streamlit_chatbot.py

import streamlit as st

st.set_page_config(page_title="Chat with the Chatbot!", page_icon="🤖")

st.title("Chat with the Chatbot! 🤖")
st.markdown("*(System message includes detailed instructions about OrderBot)*")

# Initialize chat history in session state
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "You are Service Assistant"}
    ]

# User input
user_input = st.text_input("You:", key="user_input", placeholder="Enter text here...")

# On send button click
if st.button("Send", key="send_button") and user_input:
    st.session_state.messages.append({"role": "user", "content": user_input})

    # Dummy bot response (replace with OpenAI API call for real bot)
    bot_response = f"You said: {user_input}"
    st.session_state.messages.append({"role": "assistant", "content": bot_response})

    # Clear input
    st.experimental_rerun()

# Display chat history
for msg in st.session_state.messages[1:]:  # skip system message
    if msg["role"] == "user":
        st.markdown(f"**You:** {msg['content']}")
    else:
        st.markdown(f"**Service Assistant:** {msg['content']}")



2025-10-19 17:49:07.669 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-19 17:49:07.670 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-19 17:49:07.723 
  command:

    streamlit run /Users/oliver/Documents/Systems with Openai/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-10-19 17:49:07.724 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-19 17:49:07.724 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-19 17:49:07.724 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-19 17:49:07.724 Thread 'MainThread': missing ScriptRunContext! T

In [ ]:
def get_completion_from_messages(messages):
    # Find the latest user message
    user_message = ""
    for msg in reversed(messages):
        if msg['role'] == 'user':
            user_message = msg['content'].lower()
            break

    # Simulate assistant response based on keywords
    if "hi" in user_message or "hello" in user_message:
        return "Hello! How can I assist you today?"
    elif "phone" in user_message:
        return "We offer several phones, including the SmartX Pro and TCL 503. Would you like more details?"
    elif "camera" in user_message:
        return "Our camera selection includes the FotoSnap DSLR and FotoSnap Compact. Interested in specs or pricing?"
    elif "tv" in user_message or "tvs" in user_message:
        return "We have a range of TVs, including TCL and Samsung models. What size are you looking for?"
    elif "thank" in user_message:
        return "You're welcome! If you have any more questions, just ask."
    elif "bye" in user_message:
        return "Goodbye! Have a great day."
    else:
        return "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."


In [ ]:
def evaluate_response(user_message, assistant_response):
    # Approve if the assistant response is not the fallback message
    if "connect you with a human" not in assistant_response:
        return "Yes"
    else:
        return "No"

In [ ]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    # Step 1: Moderation (simulate as always safe)
    moderation_output = {"flagged": False}
    if moderation_output["flagged"]:
        return "Sorry, we cannot process this request.", all_messages

    # Step 2: Build messages
    system_message = "You are a customer service assistant for a large electronic store."
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"}
    ]

    # Step 3: Get assistant response
    assistant_response = get_completion_from_messages(messages)
    messages.append({'role': 'assistant', 'content': assistant_response})

    # Step 4: Evaluate response
    evaluation = evaluate_response(user_input, assistant_response)
    if evaluation == "Yes":
        return assistant_response, all_messages + messages
    else:
        fallback = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return fallback, all_messages + messages


In [ ]:
import streamlit
print(streamlit.__version__)


1.50.0
